# 0.0 IMPORTS

## packages

In [1]:
import os, pickle, datetime, json
import pandas as pd




pd.options.display.max_columns = 500 # this will set limit of columns to 500
pd.set_option('display.float_format', lambda x: '%.5f' % x)

%matplotlib inline
os.getcwd()

ModuleNotFoundError: No module named 'pandas'

In [2]:
!pip install pandas

  Using cached pandas-1.5.3-cp310-cp310-win_amd64.whl (10.4 MB)
  Using cached pytz-2022.7.1-py2.py3-none-any.whl (499 kB)
  Using cached numpy-1.24.2-cp310-cp310-win_amd64.whl (14.8 MB)



[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
doed <= collec()